<a href="https://colab.research.google.com/github/Abhirrami/sdc/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install -q tensorflow gradio

# Imports
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import gradio as gr

# Constants
VOCAB_SIZE = 10000
MAX_LEN = 200

# Load dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=VOCAB_SIZE)
x_train = pad_sequences(x_train, maxlen=MAX_LEN)
x_test = pad_sequences(x_test, maxlen=MAX_LEN)

# Build LSTM model
model = Sequential([
    Embedding(VOCAB_SIZE, 64, input_length=MAX_LEN),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

# Compile and train
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=2, batch_size=128, validation_split=0.2)

# Word index for encoding custom input
word_index = imdb.get_word_index()
word_index = {k: (v + 3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

def encode_text(text):
    tokens = text.lower().split()
    encoded = [1]  # <START> token
    for word in tokens:
        encoded.append(word_index.get(word, 2))  # 2 is <UNK>
    return pad_sequences([encoded], maxlen=MAX_LEN)

# Gradio function
def predict_sentiment(text):
    encoded = encode_text(text)
    prediction = model.predict(encoded)[0][0]
    label = "Positive 😊" if prediction > 0.5 else "Negative 😞"
    return f"{label} (Confidence: {prediction * 100:.2f}%)"

# Gradio interface
gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=4, placeholder="Type a movie review here..."),
    outputs="text",
    title="LSTM Sentiment Analyzer",
    description="Enter a movie review and get a sentiment prediction using LSTM."
).launch()
